In [1]:
from counter import counter
from common_imports import *
from pathlib import Path
from csv import DictReader

In [7]:
from common_imports import *
import regex as re

In [13]:
path = "C:/Users/amcga/Downloads/econ WoS stratified top 20k"

In [8]:
#%pip install wosfile

In [9]:
stopwords = set()

def get_tups(s, whitelist=None, stopwords=None):
    
    if stopwords is None:
        stopwords = set()
    
    sp = s.lower() # debating lowercaseing..
    sp = re.sub("[^a-zA-Z\s]+", "", sp)  # removing extraneous characters
    sp = re.sub("\s+", " ", sp)  # removing extra characters
    sp = sp.strip()
    sp = sp.split()  # splitting into words

    #sp = [x for x in sp if x not in stopwords]  # strip stopwords... not yet though!
    # sp = [x for x in sp if len(x) >= 3 ] # strip short words # don't do this now, we need these for tuples.

    # print(len(tups),c['contextPure'], "---", tups)

    # keep everything in order
    tups = ["-".join(list(x)) for x in zip(sp[:-1], sp[1:])]  # two-word *ordered* tuples
    intermingle = [ sp[i//2] if i%2==0 else tups[(i-1)//2] for i in range(2*len(sp)-1) ]
    
    intermingle = [x for x in intermingle if len(x) >= 5 ] # strip short words
    if stopwords is not None:
        intermingle = [x for x in intermingle if x not in stopwords]
    if whitelist is not None:
        intermingle = [x for x in intermingle if x in whitelist]
    
    return intermingle

get_tups("This is an example of the tuples we are processing...")

['this-is',
 'is-an',
 'an-example',
 'example',
 'example-of',
 'of-the',
 'the-tuples',
 'tuples',
 'tuples-we',
 'we-are',
 'are-processing',
 'processing']

In [10]:
def count(counter, values, terms, combinations):
    
    term_pair_combinations = [x for x in combinations if ('t1' in x or 't2' in x)]
    single_term_combinations = [x for x in combinations if not ('t1' in x or 't2' in x)]
    
    for i1,t1 in enumerate(terms):
        val = dict(values)
        val['t'] = t1
        cc.cnt(val, combinations=single_term_combinations)

        if len(term_pair_combinations):
            for t2 in tups[i1+1:i1+1+250]:
                val = dict(values)
                val['t1'] = t1
                val['t2'] = t2
                cc.cnt(val, combinations=term_pair_combinations)
    

# first count

In [80]:
import glob
import wosfile
from collections import Counter

# Create a list of all relevant files. Our folder may contain multiple export files.
files = glob.glob(path + "/**/*.txt")

cc = counter()

# wosfile will read each file in the list in turn and yield each record
# for further handling
for rec in wosfile.records_from(files):
    # Records are very thin wrappers around a standard Python dict,
    # whose keys are the WoS field tags.
    # Here we look at the SC field (subject categories) and update our counter
    # with the categories in each record.

    tups = get_tups( rec.get('TI') )
    year = int(rec.get('PY'))
    journal = rec.get('SO').lower()
    
    count(
        cc,
        values = {
            'fy': year,
            'fj': journal,
        },
        terms = tups,
        combinations = [
            ('fy','t'),
            ('fj','t'),
            ('fy',),
            ('t',),
            ('fj',),
        ]
    )

Blank counter with no name


In [96]:
cc.save_counts('count-trends-top20k-strat30-WoS-econ')